### Extraindo informações mais gerais dos produtos na categoria Dermocosméticos:
* Título
* Marca
* Preço com Desconto
* Preço sem desconto

In [17]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup


chrome_options = Options()
chrome_options.add_argument("--headless") 


urls = [
    
    "https://drogariasoares.com.br/categoria/cabelo/94",
    "https://drogariasoares.com.br/categoria/bioderma-especial-novembro/172",
    "https://drogariasoares.com.br/categoria/bioderma/171",
    "https://drogariasoares.com.br/categoria/adcos/167",
    "https://drogariasoares.com.br/categoria/corpo-e-rosto/95?pagina=4",
    "https://drogariasoares.com.br/categoria/dermocosmeticos-precos-especiais/143?pagina=2",
    "https://drogariasoares.com.br/categoria/bayer-bepantol/168",
    "https://drogariasoares.com.br/categoria/isdin/169",
    "https://drogariasoares.com.br/categoria/protecao-solar/98"
]


all_products_data = []


for url in urls:
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(3)  

    
    html_content = driver.page_source

   
    driver.quit()

    
    soup = BeautifulSoup(html_content, "html.parser")

    
    produtos = soup.find_all("div", class_="item-prod")

    
    for produto in produtos:
        titulo = produto.find("h2").text.strip()
        marca = produto.find("small", class_="marca").text.strip()
        preco_sem_desconto_element = produto.find("li", class_="preco text-center").find("p", class_='preco-de')
        preco_sem_desconto = preco_sem_desconto_element.text.strip() if preco_sem_desconto_element else ""
        preco_com_desconto = produto.find("p", class_="preco-por").strong.span.text.strip()

       
        product_data = {
            "Título": titulo,
            "Marca": marca,
            "Preço sem Desconto": preco_sem_desconto,
            "Preço com Desconto": preco_com_desconto
        }
        all_products_data.append(product_data)


df1 = pd.DataFrame(all_products_data)


df1.drop_duplicates(inplace=True)



In [18]:
df1

,Título,Marca,Preço sem Desconto,Preço com Desconto
0,DUCRAY ANAPHASE SH 200ML,LABORATORIO PIERRE FABRE,,"R$ 115,99"
1,MASCARA DE TRATAMENTO PANTENE COLAGENO 550ML,PANTENE,,"R$ 32,65"
2,Serum Concentrado Hidratante e Fortalecedor Bi...,NAOS BRASIL,,"R$ 109,90"
3,AGUA MICELAR BIODERMA SEBIUM OLEOSA 100ML,NAOS BRASIL,,"R$ 45,90"
4,AGUA MICELAR BIODERMA SENSIBIO PELE SENSIVEL 2...,NAOS BRASIL,,"R$ 89,90"
...,...,...,...,...
214,ANTHELIOS FPS50 BB CREAM 40G,LOREAL BR- DIV DCA LA ROCHE,,"R$ 96,90"
215,LAKESIA ANTIMICOTICO 30ML,GENOMMA PERFUMARIA,,"R$ 46,90"
216,PROTETOR SOLAR FACIAL ACTINE DARROW FPS80 UNIV...,DARROW,,"R$ 89,90"
217,PROTETOR SOLAR FACIAL ANTHELIOS FPS60 XL PROTE...,LOREAL BR- DIV DCA LA ROCHE,,"R$ 79,90"


### Extraindo os códigos dos produtos na categoria Dermocosméticos:
* ID
* SKU
##### OBS: Infelizmente, no site da Drogaria Soares não foi localizado o código relativo ao EAN, por esse motivo, o mesmo não foi apresentado nesse código.

In [19]:
import requests
from pyquery import PyQuery as pq
import re
import concurrent.futures
import pandas as pd

def extrair_info_produto(url):
    response = requests.get(url)
    doc = pq(response.text)
    
    # Encontrar o script que contém as informações do produto
    script_content = doc('script:contains("id:")').text()
    
    # Procurar o ID e SKU do produto no conteúdo do script
    id_match = re.search(r"id:'(\d+)'", script_content)
    sku_match = re.search(r"sku:'(\d+)'", script_content)
    
    if id_match and sku_match:
        id_produto = id_match.group(1)
        sku_produto = sku_match.group(1)
        return id_produto, sku_produto
    else:
        return None, None

def extrair_info_produto_wrapper(url):
    try:
        return extrair_info_produto(url)
    except Exception as e:
        print(f"Erro ao extrair informações de {url}: {e}")
        return None, None

def extrair_links_produtos():
    urls = [
        
        "https://drogariasoares.com.br/categoria/cabelo/94",
        "https://drogariasoares.com.br/categoria/bioderma-especial-novembro/172",
        "https://drogariasoares.com.br/categoria/bioderma/171",
        "https://drogariasoares.com.br/categoria/adcos/167",
        "https://drogariasoares.com.br/categoria/corpo-e-rosto/95?pagina=4",
        "https://drogariasoares.com.br/categoria/dermocosmeticos-precos-especiais/143?pagina=2",
        "https://drogariasoares.com.br/categoria/bayer-bepantol/168",
        "https://drogariasoares.com.br/categoria/isdin/169",
        "https://drogariasoares.com.br/categoria/protecao-solar/98"
    ]
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
    }
    links = []

    for url in urls:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            doc = pq(response.text)
            products = doc('.link-prod')

            for product in products:
                href = pq(product).attr('href')
                if 'a-4-body-lotion-maca-220g' not in href:
                    links.append(href)

    return links

links_produtos = extrair_links_produtos()

dados_produtos = []
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    resultados = executor.map(extrair_info_produto_wrapper, ['https://drogariasoares.com.br' + link for link in links_produtos])
    for resultado in resultados:
        if resultado[0] is not None and resultado[1] is not None:
            dados_produtos.append(resultado)

df2 = pd.DataFrame(dados_produtos, columns=['ID do Produto', 'SKU'])
df2.drop_duplicates(inplace=True)


In [20]:
df2

,ID do Produto,SKU
0,4389256,92964
4,9992641,105364
8,4079937,98732
12,4079913,93673
16,4079934,97947
...,...,...
856,4769913,67081
860,4944202,83718
864,9991848,104572
868,4766108,99216


### Unificando e Tratando os Dataframes:

In [21]:
reindexed_df2 = df2.set_index(pd.Index(range(218)))

In [22]:

df1["ID"] = reindexed_df2["ID do Produto"]
df1["SKU"] = reindexed_df2["SKU"]

In [23]:
df1

,Título,Marca,Preço sem Desconto,Preço com Desconto,ID,SKU
0,DUCRAY ANAPHASE SH 200ML,LABORATORIO PIERRE FABRE,,"R$ 115,99",4389256,92964
1,MASCARA DE TRATAMENTO PANTENE COLAGENO 550ML,PANTENE,,"R$ 32,65",9992641,105364
2,Serum Concentrado Hidratante e Fortalecedor Bi...,NAOS BRASIL,,"R$ 109,90",4079937,98732
3,AGUA MICELAR BIODERMA SEBIUM OLEOSA 100ML,NAOS BRASIL,,"R$ 45,90",4079913,93673
4,AGUA MICELAR BIODERMA SENSIBIO PELE SENSIVEL 2...,NAOS BRASIL,,"R$ 89,90",4079934,97947
...,...,...,...,...,...,...
214,ANTHELIOS FPS50 BB CREAM 40G,LOREAL BR- DIV DCA LA ROCHE,,"R$ 96,90",4944202,83718
215,LAKESIA ANTIMICOTICO 30ML,GENOMMA PERFUMARIA,,"R$ 46,90",9991848,104572
216,PROTETOR SOLAR FACIAL ACTINE DARROW FPS80 UNIV...,DARROW,,"R$ 89,90",4766108,99216
217,PROTETOR SOLAR FACIAL ANTHELIOS FPS60 XL PROTE...,LOREAL BR- DIV DCA LA ROCHE,,"R$ 79,90",4766540,92386


In [24]:
df1.to_csv('Dermocosméticos_e_Proteção_Soares.csv')